In [1]:
!pip install ripser transformers accelerate torch numpy pandas tqdm networkx hf_transfer datasets seaborn pyarrow fastparquet datasets

In [2]:
from huggingface_hub import login
login()

In [3]:
# imports
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import numpy as np
import pandas as pd
import ripser
from ripser import ripser
from tqdm import tqdm
import math
import numpy as geek
import networkx as nx
from datasets import load_dataset
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

In [4]:
# tda functions
def find_highest_finite_value_comprehension(data):
    finite_values = [x for x in data if not math.isinf(x)]
    return max(finite_values) if finite_values else -math.inf

def get_second_value_ignoring_inf(data):
    non_inf_values = [x for x in data if not math.isinf(x)]
    if len(non_inf_values) < 2:
        return None
    return non_inf_values[1]

def get_llama_attention(text, model, tokenizer):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    
    device = next(model.parameters()).device
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = model(**inputs, output_attentions=True)
    
    attention_matrices = torch.stack(outputs.attentions).cpu()
    attention_matrix = attention_matrices.mean(dim=0).mean(dim=1).squeeze().numpy()
    np.fill_diagonal(attention_matrix, 0)
    attention_matrix = (attention_matrix + attention_matrix.T) / 2
    
    return attention_matrix

def build_graph(attention_matrix, threshold=None, percentile=50):
    graph = nx.Graph()
    num_nodes = attention_matrix.shape[0]
    
    graph.add_nodes_from(range(num_nodes))
    
    upper_triangle_indices = np.triu_indices(num_nodes, k=1)
    upper_triangle_values = attention_matrix[upper_triangle_indices]
    
    non_zero_values = upper_triangle_values[upper_triangle_values > 0]
    non_zero_values = non_zero_values[~np.isnan(non_zero_values)]
    
    if threshold is None:
        if len(non_zero_values) > 0:
            try:
                threshold = np.percentile(non_zero_values, percentile)
                if np.isnan(threshold) or threshold == 0:
                    threshold = np.max(non_zero_values) * 0.1
                else:
                    print(f"Using percentile-based threshold: {threshold:.6f} ({percentile}th percentile)")
                print(f"Non-zero values: {len(non_zero_values)}/{len(upper_triangle_values)} ({100*len(non_zero_values)/len(upper_triangle_values):.1f}%)")
                print(f"Value range: [{np.min(non_zero_values):.6f}, {np.max(non_zero_values):.6f}]")
            except:
                threshold = 0.001
                print(f"Error calculating percentile, using default threshold: {threshold}")
        else:
            threshold = 0.0
            print(f"WARNING: No non-zero attention values found!")
    else:
        print(f"Using fixed threshold: {threshold}")
    
    edge_count = 0
    for i in range(num_nodes):
        for j in range(i + 1, num_nodes):
            if attention_matrix[i, j] > threshold:
                graph.add_edge(i, j, weight=attention_matrix[i, j])
                edge_count += 1
    
    print(f"Graph created: {num_nodes} nodes, {edge_count} edges")
    
    if edge_count == 0:
        print(f"WARNING: No edges created! Threshold was {threshold:.6f}, max value was {upper_triangle_values.max():.6f}")
    
    return graph

def process_texts(texts, model_name="meta-llama/Llama-3.1-8B"):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float16,
        device_map="auto",
        low_cpu_mem_usage=True,
        attn_implementation="eager"
    )
    
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    data = []
    for text in tqdm(texts):
        try:
            attention_matrix = get_llama_attention(text, model, tokenizer)
            graph = build_graph(attention_matrix, threshold=None, percentile=50)
            tda_features = compute_tda_features(graph)
            data.append(tda_features)
        except Exception as e:
            print(f"Error processing text: {e}")
            data.append([0] * 12)

    columns = ["Num_0dim", "Max_0dim", "Max_0dim_Minus_Second", "Mean_0dim", "betti_curve_0", "persistence_entropy_0",
               "Num_1dim", "Max_1dim", "Max_1dim_Minus_Second", "Mean_1dim", "betti_curve_1", "persistence_entropy_1"]
    return pd.DataFrame(data, columns=columns)

def compute_tda_features(graph):
    adjacency_matrix = nx.to_numpy_array(graph)
    
    # Check if graph is empty or has no edges
    if adjacency_matrix.shape[0] == 0 or np.sum(adjacency_matrix) == 0:
        print("Empty graph, returning zero features")
        return [0] * 12
    
    # Use distance matrix for ripser
    distance_matrix = 1 - adjacency_matrix  # Convert similarity to distance
    np.fill_diagonal(distance_matrix, 0)  # Ensure diagonal is 0
    
    diagrams = ripser(distance_matrix, maxdim=1, distance_matrix=True)['dgms']
    
    h0 = diagrams[0]
    h1 = diagrams[1] if len(diagrams) > 1 else np.array([])

    num_h0 = geek.count_nonzero(np.round(h0))
    highest_h0 = find_highest_finite_value_comprehension(h0[:, 1] - h0[:, 0]) if num_h0 > 0 else 0
    Second_highest_h0 = get_second_value_ignoring_inf(h0[:, 1] - h0[:, 0]) if num_h0 > 1 else 0
    highest_minus_second_h0 = highest_h0 - Second_highest_h0 if num_h0 > 1 else 0

    h0[np.isinf(h0)] = 0
    mean_h0 = np.mean(h0) if num_h0 > 0 else 0

    num_h1 = geek.count_nonzero(np.round(h1))
    highest_h1 = find_highest_finite_value_comprehension(h1[:, 1] - h1[:, 0]) if num_h1 > 0 else 0
    second_highest_h1 = get_second_value_ignoring_inf(h1[:, 1] - h1[:, 0]) if num_h1 > 1 else 0
    highest_minus_second_h1 = highest_h1 - second_highest_h1 if num_h1 > 1 else 0

    h1[np.isinf(h1)] = 0
    mean_h1 = np.mean(h1) if num_h1 > 0 else 0

    h0_persistences = np.sort(h0[:, 1] - h0[:, 0]) if num_h0 > 1 else np.array([0])
    h1_persistences = np.sort(h1[:, 1] - h1[:, 0]) if num_h1 > 1 else np.array([0])

    sum_persistence_0 = np.sum(h0_persistences) if len(h0_persistences) > 0 else 0
    sum_persistence_1 = np.sum(h1_persistences) if len(h1_persistences) > 0 else 0
    persistence_entropy_0 = -np.sum(h0_persistences * np.log(h0_persistences + 1e-10)) if len(h0_persistences) > 0 else 0
    persistence_entropy_1 = -np.sum(h1_persistences * np.log(h1_persistences + 1e-10)) if len(h1_persistences) > 0 else 0
    betti_curve_0 = len(h0_persistences)
    betti_curve_1 = len(h1_persistences)

    return [num_h0, highest_h0, highest_minus_second_h0, mean_h0, betti_curve_0, persistence_entropy_0,
            num_h1, highest_h1, highest_minus_second_h1, mean_h1, betti_curve_1, persistence_entropy_1]

In [5]:
texts = ["This is a test sentence.", "Another example of text processing."]
df = process_texts(texts)
print(df)

`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

 50%|█████     | 1/2 [00:03<00:03,  3.04s/it]

Using percentile-based threshold: 0.015762 (50th percentile)
Non-zero values: 21/21 (100.0%)
Value range: [0.006226, 0.471680]
Graph created: 7 nodes, 10 edges


100%|██████████| 2/2 [00:05<00:00,  2.63s/it]

Using percentile-based threshold: 0.018997 (50th percentile)
Non-zero values: 21/21 (100.0%)
Value range: [0.008690, 0.476318]
Graph created: 7 nodes, 10 edges


   Num_0dim  Max_0dim  Max_0dim_Minus_Second  Mean_0dim  betti_curve_0  \
0         7  0.611816               0.067627   0.241752              7   
1         7  0.611816               0.065674   0.242536              7   

   persistence_entropy_0  Num_1dim  Max_1dim  Max_1dim_Minus_Second  \
0               1.933568         0         0                      0   
1               1.928764         0         0                      0   

   Mean_1dim  betti_curve_1  persistence_entropy_1  
0          0              1                   -0.0  
1          0              1                   -0.0  


In [6]:
import pyarrow
import pyarrow.parquet as pq
from datasets import load_dataset

ds = load_dataset("nyu-mll/glue", "sst2", split="train")
sst2 = ds.to_pandas()

positive_sentences = sst2[sst2['label'] == 1]['sentence'][0:500]
negative_sentences = sst2[sst2['label'] == 0]['sentence'][0:500]

print("Processing positive sentences...")
positive_tda = process_texts(positive_sentences)

print("Processing negative sentences...")
negative_tda = process_texts(negative_sentences)

sst2/train-00000-of-00001.parquet:   0%|          | 0.00/3.11M [00:00<?, ?B/s]

sst2/validation-00000-of-00001.parquet:   0%|          | 0.00/72.8k [00:00<?, ?B/s]

sst2/test-00000-of-00001.parquet:   0%|          | 0.00/148k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

...

Using percentile-based threshold: 0.023743 (50th percentile)
Non-zero values: 15/15 (100.0%)
Value range: [0.005291, 0.473389]
Graph created: 6 nodes, 7 edges


100%|█████████▉| 499/500 [29:37<00:02,  2.10s/it]

Using percentile-based threshold: 0.032127 (50th percentile)
Non-zero values: 10/10 (100.0%)
Value range: [0.007690, 0.477783]
Graph created: 5 nodes, 5 edges


100%|██████████| 500/500 [29:39<00:00,  3.56s/it]

Using percentile-based threshold: 0.012787 (50th percentile)
Non-zero values: 78/78 (100.0%)
Value range: [0.001736, 0.473145]
Graph created: 13 nodes, 39 edges


In [7]:
!pip install textdescriptives spacy
!python -m spacy download en_core_web_sm

import spacy
import textdescriptives as td
from tqdm import tqdm
import pandas as pd

# Load spacy model
nlp = spacy.load("en_core_web_sm")
nlp.add_pipe("textdescriptives/all")

def get_desc(data):
    """Extract linguistic features from text data"""
    doc = nlp(data.iloc[0])
    df1 = td.extract_df(doc)
    
    for text in tqdm(data.iloc[1:]):
        text = nlp(text)
        df2 = td.extract_df(text)
        df1 = pd.concat([df1, df2], ignore_index=True)
    
    return df1

print("Extracting linguistic features for positive sentences...")
positive_feat = get_desc(positive_sentences)

print("Extracting linguistic features for negative sentences...")
negative_feat = get_desc(negative_sentences)

feature_categories = {
    'token_stats': [
        'token_length_mean', 'token_length_median', 'token_length_std',
        'n_tokens', 'n_unique_tokens', 'proportion_unique_tokens',
        'mean_word_length', 'alpha_ratio'
    ],
    
    'sentence_stats': [
        'sentence_length_mean', 'sentence_length_median', 'sentence_length_std',
        'n_sentences', 'doc_length'
    ],
    
    'syllable_stats': [
        'syllables_per_token_mean', 'syllables_per_token_median', 
        'syllables_per_token_std'
    ],
    
    'pos_proportions': [
        'pos_prop_ADJ', 'pos_prop_ADP', 'pos_prop_ADV', 'pos_prop_AUX',
        'pos_prop_CCONJ', 'pos_prop_DET', 'pos_prop_INTJ', 'pos_prop_NOUN',
        'pos_prop_NUM', 'pos_prop_PART', 'pos_prop_PRON', 'pos_prop_PROPN',
        'pos_prop_PUNCT', 'pos_prop_SCONJ', 'pos_prop_SYM', 'pos_prop_VERB',
        'pos_prop_X'
    ],
    
    'information_theory': [
        'entropy', 'perplexity', 'per_word_perplexity', 'oov_ratio'
    ],
    
    'coherence': [
        'first_order_coherence', 'second_order_coherence'
    ],
    
    'dependency': [
        'dependency_distance_mean', 'dependency_distance_std',
        'prop_adjacent_dependency_relation_mean', 
        'prop_adjacent_dependency_relation_std'
    ],
    
    'readability': [
        'flesch_reading_ease', 'flesch_kincaid_grade', 'smog',
        'gunning_fog', 'automated_readability_index', 
        'coleman_liau_index', 'lix', 'rix'
    ],
    
    'quality_duplicates': [
        'passed_quality_check', 'duplicate_line_chr_fraction',
        'duplicate_paragraph_chr_fraction', 'duplicate_ngram_chr_fraction_5',
        'duplicate_ngram_chr_fraction_6', 'duplicate_ngram_chr_fraction_7',
        'duplicate_ngram_chr_fraction_8', 'duplicate_ngram_chr_fraction_9',
        'duplicate_ngram_chr_fraction_10'
    ],
    
    'ngram_stats': [
        'top_ngram_chr_fraction_2', 'top_ngram_chr_fraction_3',
        'top_ngram_chr_fraction_4'
    ],
    
    'formatting': [
        'n_characters', 'n_stop_words', 'symbol_to_word_ratio_#',
        'proportion_ellipsis', 'proportion_bullet_points', 
        'contains_lorem ipsum'
    ]
}

def get_category_features(df, category_name, feature_list):
    available_features = [f for f in feature_list if f in df.columns]
    return df[available_features]

positive_categorized = {}
negative_categorized = {}

for category, features in feature_categories.items():
    positive_categorized[category] = get_category_features(
        positive_feat, category, features
    )
    negative_categorized[category] = get_category_features(
        negative_feat, category, features
    )
    print(f"{category}: {len(positive_categorized[category].columns)} features")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 39.5 MB/s  0:00:00 eta 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
Extracting linguistic features for positive sentences...


  2%|▏         | 11/499 [00:00<00:04, 109.01it/s]/usr/local/lib/python3.12/dist-packages/textdescriptives/components/coherence.py:44: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Span.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  similarities.append(sent.similarity(sents[i + order]))
100%|██████████| 499/499 [00:03<00:00, 127.03it/s]


Extracting linguistic features for negative sentences...


100%|██████████| 499/499 [00:03<00:00, 130.55it/s]

token_stats: 8 features
sentence_stats: 5 features
syllable_stats: 3 features
pos_proportions: 17 features
information_theory: 4 features
coherence: 2 features
dependency: 4 features
readability: 8 features
quality_duplicates: 9 features
ngram_stats: 3 features
formatting: 6 features
